In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [5]:
figuras = []


def obter_parametros_onda(numero):
    if numero == 1:
        frequencia = float(entry_frequencia1.get())
        amplitude = float(entry_amplitude1.get())
        fase = float(entry_fase1.get())
    elif numero == 2:
        frequencia = float(entry_frequencia2.get())
        amplitude = float(entry_amplitude2.get())
        fase = float(entry_fase2.get())
    return frequencia, amplitude, fase


def plotar_grafico_onda(t, ondas, titulo, cores, infos, linhas, opacidade, container):
    for widget in container.winfo_children():
        widget.destroy()

    fig = Figure(figsize=(8, 4))
    ax = fig.add_subplot(111)

    for i, onda in enumerate(ondas):
        ax.plot(
            t,
            onda,
            color=cores[i],
            label=infos[i],
            linestyle=linhas[i],
            alpha=opacidade[i],
        )

    ax.set_title(titulo)
    ax.set_xlabel("Tempo")
    ax.set_ylabel("Amplitude")
    ax.grid(True)
    ax.legend()

    canvas = FigureCanvasTkAgg(fig, master=container)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    # Adicionar a figura à lista
    figuras.append(fig)


def plotar_graficos():
    try:
        figuras.clear()

        frequencia1, amplitude1, fase1 = obter_parametros_onda(1)
        frequencia2, amplitude2, fase2 = obter_parametros_onda(2)

        t = np.linspace(0, 2 * np.pi, 1000)

        onda1 = amplitude1 * np.sin(2 * np.pi * frequencia1 * t + fase1)
        onda2 = amplitude2 * np.sin(2 * np.pi * frequencia2 * t + fase2)

        interferencia_construtiva = onda1 + onda2
        interferencia_destrutiva = onda1 - onda2

        info_onda1 = (
            f"Amplitude: {amplitude1}\nFrequência: {frequencia1} Hz\nFase: {fase1} rad"
        )
        info_onda2 = (
            f"Amplitude: {amplitude2}\nFrequência: {frequencia2} Hz\nFase: {fase2} rad"
        )
        info_interf_construtiva = "Interferência Construtiva"
        info_interf_destrutiva = "Interferência Destrutiva"

        ondas = [onda1, onda2, interferencia_construtiva, interferencia_destrutiva]
        cores = ["blue", "orange", "green", "red"]
        linhas = ["-", "-", "--", "--"]
        opacidades = [1, 1, 0.25, 0.25]
        infos = [
            info_onda1,
            info_onda2,
            info_interf_construtiva,
            info_interf_destrutiva,
        ]

        plotar_grafico_onda(
            t,
            [ondas[0]],
            "Onda 1",
            [cores[0]],
            [infos[0]],
            [linhas[0]],
            [opacidades[0]],
            frame_onda1,
        )

        plotar_grafico_onda(
            t,
            [ondas[1]],
            "Onda 2",
            [cores[1]],
            [infos[1]],
            [linhas[1]],
            [opacidades[1]],
            frame_onda2,
        )

        plotar_grafico_onda(
            t,
            [ondas[2]],
            "Interferência Construtiva",
            [cores[2]],
            [infos[2]],
            [linhas[2]],
            [opacidades[2]],
            frame_interf_construtiva,
        )

        plotar_grafico_onda(
            t,
            [ondas[3]],
            "Interferência Destrutiva",
            [cores[3]],
            [infos[3]],
            [linhas[3]],
            [opacidades[3]],
            frame_interf_destrutiva,
        )

        plotar_grafico_onda(
            t,
            ondas,
            "Todas as Ondas",
            cores,
            infos,
            linhas,
            opacidades,
            frame_todas_as_ondas,
        )

        frame_principal.update_idletasks()
        canvas.configure(scrollregion=canvas.bbox("all"))

    except ValueError:
        messagebox.showerror(
            "Erro", "Certifique-se de preencher todos os campos com valores numéricos."
        )


def salvar_todos_pdfs():
    nome_arquivo = filedialog.asksaveasfilename(
        defaultextension=".pdf", filetypes=[("Arquivo PDF", "*.pdf")]
    )
    if nome_arquivo:
        with PdfPages(nome_arquivo) as pdf:
            for fig in figuras:
                pdf.savefig(fig)


def reiniciar_aplicacao():
    for widget in root.winfo_children():
        widget.destroy()

    criar_interface()


def criar_interface():
    global entry_frequencia1, entry_amplitude1, entry_fase1
    global entry_frequencia2, entry_amplitude2, entry_fase2
    global frame_principal, canvas, frame_graficos
    global frame_onda1, frame_onda2, frame_interf_construtiva, frame_interf_destrutiva, frame_todas_as_ondas

    frame_principal = tk.Frame(root)
    frame_principal.pack(fill=tk.BOTH, expand=1)

    canvas = tk.Canvas(frame_principal)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

    scrollbar = tk.Scrollbar(frame_principal, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    canvas.configure(yscrollcommand=scrollbar.set)

    frame_graficos = tk.Frame(canvas)
    canvas.create_window((0, 0), window=frame_graficos, anchor=tk.NW)

    frame_onda1 = tk.Frame(frame_graficos, padx=10, pady=10)
    frame_onda1.grid(row=0, column=0, padx=10, pady=10)

    label_frequencia1 = tk.Label(frame_onda1, text="Frequência (Hz) Onda 1:")
    label_frequencia1.grid(row=0, column=0, padx=5, pady=5)
    entry_frequencia1 = tk.Entry(frame_onda1)
    entry_frequencia1.grid(row=0, column=1)

    label_amplitude1 = tk.Label(frame_onda1, text="Amplitude Onda 1:")
    label_amplitude1.grid(row=1, column=0, padx=5, pady=5)
    entry_amplitude1 = tk.Entry(frame_onda1)
    entry_amplitude1.grid(row=1, column=1)

    label_fase1 = tk.Label(frame_onda1, text="Fase (radianos) Onda 1:")
    label_fase1.grid(row=2, column=0, padx=5, pady=5)
    entry_fase1 = tk.Entry(frame_onda1)
    entry_fase1.grid(row=2, column=1)

    frame_onda2 = tk.Frame(frame_graficos, padx=10, pady=10)
    frame_onda2.grid(row=1, column=0, padx=10, pady=10)

    label_frequencia2 = tk.Label(frame_onda2, text="Frequência (Hz) Onda 2:")
    label_frequencia2.grid(row=0, column=0, padx=5, pady=5)
    entry_frequencia2 = tk.Entry(frame_onda2)
    entry_frequencia2.grid(row=0, column=1)

    label_amplitude2 = tk.Label(frame_onda2, text="Amplitude Onda 2:")
    label_amplitude2.grid(row=1, column=0, padx=5, pady=5)
    entry_amplitude2 = tk.Entry(frame_onda2)
    entry_amplitude2.grid(row=1, column=1)

    label_fase2 = tk.Label(frame_onda2, text="Fase (radianos) Onda 2:")
    label_fase2.grid(row=2, column=0, padx=5, pady=5)
    entry_fase2 = tk.Entry(frame_onda2)
    entry_fase2.grid(row=2, column=1)

    frame_interf_construtiva = tk.Frame(frame_graficos, padx=10, pady=10)
    frame_interf_construtiva.grid(row=2, column=0, padx=10, pady=10)

    frame_interf_destrutiva = tk.Frame(frame_graficos, padx=10, pady=10)
    frame_interf_destrutiva.grid(row=3, column=0, padx=10, pady=10)

    frame_todas_as_ondas = tk.Frame(frame_graficos, padx=10, pady=10)
    frame_todas_as_ondas.grid(row=4, column=0, padx=10, pady=10)

    botao_plotar = tk.Button(root, text="Plotar Gráficos", command=plotar_graficos)
    botao_plotar.pack(pady=10)

    botao_salvar_todos = tk.Button(
        root, text="Salvar Todos os Gráficos em PDF", command=salvar_todos_pdfs
    )
    botao_salvar_todos.pack(pady=10)

    botao_reiniciar = tk.Button(root, text="Reiniciar", command=reiniciar_aplicacao)
    botao_reiniciar.pack(pady=10)


root = tk.Tk()
root.title("Simulador de Ondas")
root.geometry("1000x800")

criar_interface()

root.mainloop()